In [1]:
#Library 예시

import math
import time
from itertools import chain
import argparse
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR
# 모델 학습을 위해 CUDA 환경 설정. : 지피유 설정
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
#device=torch.device('cpu')
torch.cuda.is_available()

True

In [2]:
wavenumbers=np.linspace(1000,8000,141)

In [3]:
# 별도의 데이터 Pre-Processing 과정은 없고 모델 훈련시 검증을 위해 train 중 10000개를 validation 용으로 분리.
# 새로 만든 train.csv는 train1.csv, validation은 val.csv로 저장.
# dataframe.sample(frac=1) 을 통해 셔플.
path_train = 'train_parameter.csv'
#path_test = 'test.csv'
layers = [['e_inf','w01','wp1','gamma1','w02','wp2','gamma2','thickness(nm)','level'], [str(i) for i in wavenumbers.tolist()]]
layers = list(chain(*layers))

train = pd.read_csv(path_train)
print(train.shape)
train = train.sample(frac=1)
rows, cols = train.shape

train1 = train.iloc[:rows - 20255,:]
train1 = train1.values
train1 = pd.DataFrame(data=train1,columns=layers)
train1.to_csv('train1_parameter.csv', index_label='id')

print("train file saved....")
val = train.iloc[rows - 20255:,:]
val = val.values
val = pd.DataFrame(data=val,columns=layers)
val.to_csv('val_parameter.csv', index_label='id')

(1020255, 150)
train file saved....


In [4]:
# 새로 만든 train/ val 모델 학습 데이터 경로를 설정.
train_path = 'train1_parameter.csv'#레이어를 같게 두고 풀때
val_path = 'val_parameter.csv'

lr = 1e-03
adam_epsilon = 1e-06
epochs = 100
batch_size = 2048
warmup_step = 2000
#loss_fn = nn.L1Loss()
loss_fn = nn.MSELoss()
class PandasDataset(Dataset):
    def __init__(self, path):
        super(PandasDataset, self).__init__()
        train = pd.read_csv(path).iloc[:,1:]
        self.train_X, self.train_Y = train.iloc[:,9:], train.iloc[:,0:9]
        self.tmp_x , self.tmp_y = self.train_X.values, self.train_Y.values
    
    def __len__(self):
        return len(self.train_X)

    def __getitem__(self, idx):
        return {
            'X':torch.from_numpy(self.tmp_x)[idx],
            'Y':torch.from_numpy(self.tmp_y)[idx]
        }
            
train_dataset = PandasDataset(train_path)
train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, num_workers=0)#card에 data size/batch size만큼 monunt

val_dataset = PandasDataset(val_path)
val_loader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True, num_workers=0) 

In [5]:
print(len(train_loader))

489


In [6]:
# 4개의 layer_1, layer_2, layer_3, layer_4의 두께를 구하는 방법이라 마지막 Dense unit을 1로
# 해서 각각의 두께를 구하는 모델을 구성해보았으나 결과는 좋지 않음.
# 특별한 탐색적 자료 분석 없음.
"""
- Modeling

Learning rate : 다양한 learning rate 스케줄링. Warmup 스케줄링이 train, val loss 줄이는데 가장 효과적.
Activation function : ReLU, ELU, CeLU, GeLU 사용. 음의 영역에서는 0의 값을 가지는 relu 대신 ELU, CeLU, GeLU등을 사용. relu보다는 효과적.
Optimizer : adam, adamW.
Batch size : batch size는 10000으로 시작하였으나 10000보다 작은 값에서 효과적.
Hidden node, layer depth : node 수는 Nh = Ns/(a * (Ni + No))에 따라 초기 모델링에서는 약 2500개 정도로 하였으나 실험적으로 많은
node가 train, val loss 줄이는데 효과적.
(Nh : number of hidden , Ni : Number of input, No : Number of output, a : 2 - 10)
Loss funciotion : L1 Loss
BatchNorm : 모든 layer에 batchNorm 적용
MLP Model : 단순한 MLP 모델로는 한계를 느껴 MLP의 복잡성을 증가시키는 방향으로 설계
        (1) 노드 수를 증가시켰다 줄이는 방식(UP-block, DOWN-block)
        (2) Down-block에서 skip connection과 layer norm 적용
"""

'\n- Modeling\n\nLearning rate : 다양한 learning rate 스케줄링. Warmup 스케줄링이 train, val loss 줄이는데 가장 효과적.\nActivation function : ReLU, ELU, CeLU, GeLU 사용. 음의 영역에서는 0의 값을 가지는 relu 대신 ELU, CeLU, GeLU등을 사용. relu보다는 효과적.\nOptimizer : adam, adamW.\nBatch size : batch size는 10000으로 시작하였으나 10000보다 작은 값에서 효과적.\nHidden node, layer depth : node 수는 Nh = Ns/(a * (Ni + No))에 따라 초기 모델링에서는 약 2500개 정도로 하였으나 실험적으로 많은\nnode가 train, val loss 줄이는데 효과적.\n(Nh : number of hidden , Ni : Number of input, No : Number of output, a : 2 - 10)\nLoss funciotion : L1 Loss\nBatchNorm : 모든 layer에 batchNorm 적용\nMLP Model : 단순한 MLP 모델로는 한계를 느껴 MLP의 복잡성을 증가시키는 방향으로 설계\n        (1) 노드 수를 증가시켰다 줄이는 방식(UP-block, DOWN-block)\n        (2) Down-block에서 skip connection과 layer norm 적용\n'

In [7]:
loss_fn=nn.L1Loss()
loss_fn

L1Loss()

In [46]:
class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
class LayerNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-5):
        """Construct a layernorm module in the TF style (epsilon inside the square root).
        """
        super(LayerNorm, self).__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps

        self.init_weights()

    def init_weights(self):
        self.weight.data.fill_(1.0)
        self.bias.data.zero_()

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.weight * x + self.bias
class skipConnectionModel(nn.Module):
    def __init__(self):
        super(skipConnectionModel, self).__init__()
        
        self.ln = LayerNorm(10000)
        self.ln1 = LayerNorm(7000)
        self.ln2 = LayerNorm(4000)
        self.ln3 = LayerNorm(2000)
        
        self.upblock1 = nn.Sequential(nn.Linear(141, 2000),GELU(),nn.BatchNorm1d(2000))
        self.upblock2 = nn.Sequential(nn.Linear(2000,4000),GELU(),nn.BatchNorm1d(4000))
        self.upblock3 = nn.Sequential(nn.Linear(4000,7000), GELU(),nn.BatchNorm1d(7000))
        self.upblock4 = nn.Sequential(nn.Linear(7000,10000),GELU(),nn.BatchNorm1d(10000))

        self.downblock1 = nn.Sequential(nn.Linear(10000, 7000),GELU(),nn.BatchNorm1d(7000))
        self.downblock2 = nn.Sequential(nn.Linear(7000, 4000),GELU(),nn.BatchNorm1d(4000))
        self.downblock3 = nn.Sequential(nn.Linear(4000, 2000),GELU(),nn.BatchNorm1d(2000))
        self.downblock4 = nn.Sequential(nn.Linear(2000, 300),GELU(),nn.BatchNorm1d(300))
        
        self.fclayer = nn.Sequential(nn.Linear(300,9))
#         self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        upblock1_out = self.upblock1(x)
        upblock2_out = self.upblock2(upblock1_out)
        upblock3_out = self.upblock3(upblock2_out)
        upblock4_out = self.upblock4(upblock3_out)
        
        downblock1_out = self.downblock1(self.ln(upblock4_out))
        skipblock1 = downblock1_out + upblock3_out
        downblock2_out = self.downblock2(self.ln1(skipblock1))
        skipblock2 = downblock2_out + upblock2_out
        downblock3_out = self.downblock3(self.ln2(skipblock2))
        skipblock3 = downblock3_out + upblock1_out
        downblock4_out = self.downblock4(self.ln3(skipblock3))
        
        output = self.fclayer(downblock4_out)
        
        return output
    
def get_constant_schedule(optimizer, last_epoch=-1):
    """ Create a schedule with a constant learning rate.
    """
    return LambdaLR(optimizer, lambda _: 1, last_epoch=last_epoch)


def get_constant_schedule_with_warmup(optimizer, num_warmup_steps, last_epoch=-1):
    """ Create a schedule with a constant learning rate preceded by a warmup
    period during which the learning rate increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1.0, num_warmup_steps))
        return 1.0

    return LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)


def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, last_epoch=-1):
    """ Create a schedule with a learning rate that decreases linearly after
    linearly increasing during a warmup period.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)


def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, last_epoch=-1):
    """ Create a schedule with a learning rate that decreases following the
    values of the cosine function between 0 and `pi * cycles` after a warmup
    period during which it increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)


def get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps, num_cycles=1.0, last_epoch=-1
):
    """ Create a schedule with a learning rate that decreases following the
    values of the cosine function with several hard restarts, after a warmup
    period during which it increases linearly between 0 and 1.
    """

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        if progress >= 1.0:
            return 0.0
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * ((float(num_cycles) * progress) % 1.0))))

    return LambdaLR(optimizer, lr_lambda, last_epoch)
class AdamW(Optimizer):
    """ Implements Adam algorithm with weight decay fix.
    Parameters:
        lr (float): learning rate. Default 1e-3.
        betas (tuple of 2 floats): Adams beta parameters (b1, b2). Default: (0.9, 0.999)
        eps (float): Adams epsilon. Default: 1e-6
        weight_decay (float): Weight decay. Default: 0.0
        correct_bias (bool): can be set to False to avoid correcting bias in Adam (e.g. like in Bert TF repository). Default True.
    """

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.0, correct_bias=True):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias)
        super().__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state["step"] = 0
                    # Exponential moving average of gradient values
                    state["exp_avg"] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state["exp_avg_sq"] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                beta1, beta2 = group["betas"]

                state["step"] += 1

                # Decay the first and second moment running average coefficient
                # In-place operations to update the averages at the same time
                exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1.0 - beta2, grad, grad)
                denom = exp_avg_sq.sqrt().add_(group["eps"])

                step_size = group["lr"]
                if group["correct_bias"]:  # No bias correction for Bert
                    bias_correction1 = 1.0 - beta1 ** state["step"]
                    bias_correction2 = 1.0 - beta2 ** state["step"]
                    step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom)

                # Just adding the square of the weights to the loss function is *not*
                # the correct way of using L2 regularization/weight decay with Adam,
                # since that will interact with the m and v parameters in strange ways.
                #
                # Instead we want to decay the weights in a manner that doesn't interact
                # with the m/v parameters. This is equivalent to adding the square
                # of the weights to the loss with plain (non-momentum) SGD.
                # Add weight decay at the end (fixed version)
                if group["weight_decay"] > 0.0:
                    p.data.add_(-group["lr"] * group["weight_decay"], p.data)

        return loss    

In [47]:
model = skipConnectionModel()
model = model.to(device) # 모델을 GPU 메모리에 올림.

In [49]:
#0104 100번 학습 추가
#0107 400번 학습___끝.

In [50]:
"""
모델 학습
"""

total_step = len(train_loader) * epochs
print(f"Total step is....{total_step}") # 모델이 학습하는 전체 step 계산.

# 옵티마이저와 스케줄러의 파라미터들을 정의.

no_decay = ["bias", "LayerNorm.weight"] # decay하지 않을 영역 지정.
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=adam_epsilon)
scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_step, num_training_steps=total_step
)

# train loss와 val loss, thick_loss 지정.
total_loss = 0.0
total_val_loss = 0.0
total_thick_loss=0.0
total_level_loss=0.0
# 모델 이름을 위해서 변수 만듦.
version = time.localtime()[3:5]
curr_lr = lr

n_val_loss = 10000000. # 가장 낮은 validation loss를 저장하기 위해서 변수 설정.
train_loss_epoch=[]
thick_loss_epoch=[]
level_loss_epoch=[]
val_loss_epoch=[]

start = time.time() #학습시간
for epoch in range(epochs):
    total_loss = 0 
    total_val_loss = 0
    total_thik_loss=0
    total_level_loss=0
    for i, data in enumerate(tqdm(train_loader, desc='*********Train mode*******')):  # train 데이터를 부르고 학습.
        # forward pass
        pred = model(data['X'].float().to(device))
        loss = loss_fn(pred, data['Y'].float().to(device))
        
        # backward pass
        optimizer.zero_grad() # optimizer 객체 사용해서 학습 가능한 가중치 변수에 대한 모든 변화도를 0으로 만듦
        loss.backward() 
        optimizer.step() # update optimizer params
        scheduler.step() # update scheduler params
        
        total_loss += loss.item()
        
    train_loss = total_loss / len(train_loader)
    train_loss_epoch.append(train_loss)
    print ("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch+1, epochs, train_loss))
    print('Train Time: ',time.time()-start)

    # evaluation
    # validation 데이터를 부르고 epoch 마다 학습된 모델을 부르고 평가.
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(tqdm(val_loader, desc='*********Evaluation mode*******')):
            pred = model(data['X'].float().to(device))
            loss_val = loss_fn(pred, data['Y'].float().to(device))
#             loss_thick=loss_fn(pred[:,0],data['Y'].float().to(device)[:,0])
#             loss_level=loss_fn(pred[:,1],data['Y'].float().to(device)[:,1])
            #print(pred[:,:])
            #print(data['Y'].float().to(device))
            total_val_loss += loss_val.item()
#             total_thick_loss = loss_thick.item()
#             total_level_loss = loss_level.item()
    val_loss = total_val_loss / len(val_loader)
#     thick_loss = total_thick_loss / len(val_loader)
#     level_loss = total_level_loss/len(val_loader)
    val_loss_epoch.append(val_loss)
#     thick_loss_epoch.append(thick_loss)
#     level_loss_epoch.append(level_loss)
    print ("Epoch [{}/{}], Eval Loss: {:.4f}".format(epoch+1, epochs, val_loss))
    
    # best model을 저장.
    if val_loss < n_val_loss:
        n_val_loss = val_loss
        torch.save(model.state_dict(), f'test_percent_unit_{version}_{lr}_{epochs}_MAE.pth')
        print("Best Model saved......")

Total step is....48900



Epoch [1/100], Train Loss: 366.7264
Train Time:  156.10077714920044



Epoch [1/100], Eval Loss: 363.5174
Best Model saved......



Epoch [2/100], Train Loss: 188.1636
Train Time:  322.55453848838806



Epoch [2/100], Eval Loss: 176.7571
Best Model saved......


...


Epoch [98/100], Eval Loss: 12.3121
Best Model saved......



Epoch [99/100], Train Loss: 12.0100
Train Time:  16176.129212379456



Epoch [99/100], Eval Loss: 12.2937
Best Model saved......



Epoch [100/100], Train Loss: 11.9959
Train Time:  16345.647074222565



Epoch [100/100], Eval Loss: 12.2891
Best Model saved......


In [32]:
loss_array=np.array(thick_loss_epoch)
loss_array2=np.array(val_loss_epoch)
loss_array3=np.array(level_loss_epoch)
loss_array4=np.array(train_loss_epoch)
thick_loss_data=pd.DataFrame(loss_array)
val_loss_data=pd.DataFrame(loss_array2)
level_loss_data=pd.DataFrame(loss_array3)
train_loss_data=pd.DataFrame(loss_array4)
thick_loss_data.to_csv(f'MAE_thick_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')#level을 동일하게 놔두면 평균 1nm까지 오차가 줄어든다.
val_loss_data.to_csv(f'MAE_val_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')
level_loss_data.to_csv(f'MAE_level_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')
train_loss_data.to_csv(f'MAE_train_same_layer_percent_unit_dropout=0.1_loss_lr={lr}_epoch={epochs}_level.csv', index='epochs')

In [33]:
thick_loss_data.tail()

,0
95,0.000645
96,0.000633
97,0.000628
98,0.000626
99,0.000625


In [13]:
level_loss_data.tail()

,0
95,0.000416
96,0.000406
97,0.000394
98,0.000396
99,0.000392


In [18]:
test_file='R_test_para.xlsx'
test=pd.read_excel(test_file)
test

,Unnamed: 0,1000,1050,1100,1150,1200,1250,1300,1350,1400,...,7550,7600,7650,7700,7750,7800,7850,7900,7950,8000
0,0,0.43274,0.56493,0.67806,0.52101,0.48143,0.42936,0.47108,0.46866,0.45821,...,0.10975,0.11241,0.11412,0.11640,0.11821,0.12053,0.12286,0.12482,0.12722,0.12928
1,1,0.43904,0.56862,0.69203,0.53002,0.48344,0.43182,0.47971,0.47661,0.47004,...,0.11587,0.11831,0.12015,0.12254,0.12504,0.12752,0.12969,0.13195,0.13482,0.13703
2,2,0.44766,0.57542,0.68836,0.53012,0.48931,0.44098,0.48630,0.48413,0.47552,...,0.10345,0.10554,0.10711,0.10924,0.11136,0.11369,0.11561,0.11785,0.11992,0.12265
3,3,0.44381,0.57365,0.68132,0.52431,0.48528,0.43649,0.47964,0.47662,0.47057,...,0.09973,0.10198,0.10312,0.10493,0.10714,0.10924,0.11082,0.11260,0.11502,0.11727
4,4,0.45495,0.58398,0.69344,0.53566,0.49572,0.44718,0.49124,0.48895,0.48152,...,0.10558,0.10739,0.10925,0.11054,0.11234,0.11425,0.11613,0.11763,0.11949,0.12158
5,5,0.44943,0.58195,0.69813,0.53616,0.49443,0.44360,0.48862,0.48603,0.47711,...,0.10776,0.10986,0.11194,0.11422,0.11629,0.11880,0.12101,0.12293,0.12548,0.12778


In [41]:
"""
모델 테스트
"""

test_model = skipConnectionModel()

# test 파일 경로 및 test 데이터 로드
path_test = 'R_test_para.xlsx'
#path_test='val_same_layer_level.csv'
class TestDataset(Dataset):
    def __init__(self, path_test):
        super(TestDataset, self).__init__()
        test = pd.read_excel(path_test)
        self.test_X = test.iloc[:,1:]
        self.tmp_x = self.test_X.values
    
    def __len__(self):
        return len(self.test_X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.tmp_x)[idx]
    
test_data = TestDataset(path_test)
test_loader = DataLoader(test_data, batch_size=6,pin_memory=True, num_workers=0)

In [42]:
# 모델에 학습된 가중치를 업로드.
weights = torch.load(f'test_percent_unit_{version}_{lr}_{epochs}_MAE.pth', map_location='cuda:0')
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()
test_time=time.time()
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        outputs = test_model(data.float())
    print('Test Time : ',time.time()-test_time)
pred_test = outputs

Test Time :  0.0


In [43]:
#test_layer=[[str(i) for i in np.arange(1000,8100,100).tolist()],[str(i) for i in np.arange(1000,8100,100).tolist()],['layer(nm)','level(%)']]
test_layer=['e_inf','w01','wp1','gamma1','w02','wp2','gamma2','thickness(nm)','level']
# test_layer=list(chain(*test_layer))

In [44]:
sub=pred_test.cpu().numpy()
sub=pd.DataFrame(data=sub, columns=test_layer)
sub.to_csv('result_same_layer_para.csv',index='id')

In [45]:
sub

,e_inf,w01,wp1,gamma1,w02,wp2,gamma2,thickness(nm),level
0,2.125754,1542.597900,282.905182,6.644070,1018.140625,915.304993,14.726528,282.176300,-0.229972
1,2.131060,1490.769653,98.949913,19.234056,1083.732544,933.583740,11.568529,278.520538,0.736899
2,2.215939,1390.298584,49.324501,21.253540,1099.122681,933.841431,10.632383,261.401703,3.393048
3,2.191334,1840.561035,99.566002,14.809834,1090.087402,958.431946,13.148516,261.583954,1.228789
4,2.183171,1346.465820,38.456215,24.346054,1087.763550,927.077515,11.410627,258.700958,3.577882
5,2.232140,1346.126709,80.203743,21.461794,1083.693848,923.444092,11.206762,264.795990,3.159704


In [22]:
for i in range(len(sub)):
    sub.iloc[i,1]=sub.iloc[i,1]/1000
sub.to_csv('result_same_layer_30pt.csv',index=None)

In [23]:
sub

,thick(nm),level
0,311.410736,0.007116
1,311.686951,0.007404
2,311.945740,0.007302
3,312.254181,0.007216
4,312.426849,0.007593
5,312.680786,0.007542
6,312.777466,0.008309
7,313.088501,0.008370
8,313.330872,0.008596
9,313.388611,0.009028


In [ ]:
#percent_unit: 최대 0

In [13]:
"""
모델 테스트
"""

test_model = skipConnectionModel()

# test 파일 경로 및 test 데이터 로드
path_test = 'test.csv'
class TestDataset(Dataset):
    def __init__(self, path_test):
        super(TestDataset, self).__init__()
        test = pd.read_csv(path_test)
        self.test_X = test.iloc[:,1:]
        self.tmp_x = self.test_X.values
    
    def __len__(self):
        return len(self.test_X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.tmp_x)[idx]
    
test_data = TestDataset(path_test)
test_loader = DataLoader(test_data, batch_size=10000,  num_workers=0)

# 모델에 학습된 가중치를 업로드.
weights = torch.load(f'test_{version}_{lr}_{epochs}.pth', map_location='cuda:0')
test_model.load_state_dict(weights)
test_model = test_model.to(device)
test_model.eval()

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        outputs = test_model(data.float())
pred_test = outputs

sample_sub = pd.read_csv('sample_submission.csv', index_col=0)
layers = ['layer_1','layer_2','layer_3','layer_4']
submission = sample_sub.values + pred_test.cpu().numpy()

submission = pd.DataFrame(data=submission,columns=layers)
submission.to_csv(f'test_{version}_{lr}_{epochs}.csv', index_label='id')

In [14]:
a=pd.read_csv('test_(4, 43)_0.001_100.csv')S
a

,id,layer_1,layer_2,layer_3,layer_4
0,0,255.066818,229.538574,131.793243,85.331375
1,1,158.440231,127.189491,235.908920,99.370300
2,2,148.246796,179.421295,273.704498,156.416672
3,3,91.224670,229.747925,188.885864,82.969620
4,4,273.475708,295.184357,244.910355,271.351715
5,5,99.921021,222.285248,270.671722,207.755157
6,6,147.663086,39.443428,135.538361,187.207535
7,7,274.795715,290.746735,148.076370,260.887665
8,8,85.273628,244.025543,170.481339,12.544601
9,9,218.571198,125.653633,249.992706,204.116394
